In [20]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import interp1d
from scipy import stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

import utils

# Read data
## 1. Period level

In [21]:
data_file = './data/paleo_meso_ceno_data_paleogeography_epoch.csv'

data = pd.read_csv(data_file)
data = data.sort_values(by=['mean_ma'], ascending=False)

time_file = './data/timeScale.xlsx'
time = pd.read_excel(time_file,sheet_name='Sheet1')
time_period = utils.get_timesubset(time,3,541)
width_period = [time_period['max_ma'][i] - time_period['min_ma'][i] for i in list(time_period.index)]

time_epoch = utils.get_timesubset(time,4,541)
width_epoch = [time_epoch['max_ma'][i] - time_epoch['min_ma'][i] for i in list(time_epoch.index)]


In [22]:
data_skeletal_period = utils.get_stats(data,'skeletal_total','period')
data_skeletal_period = data_skeletal_period.sort_values(by=['time'], ascending=True)

data_animal_period = utils.get_stats(data,'total_animals_new','period')
data_animal_period = data_animal_period.sort_values(by=['time'], ascending=True)

# Load Sepkoski data

In [23]:
sepkoski_period = pd.read_csv('./data/SQS_diversity_data/sepkoski_diversity_period_level.csv')

# Diversity

In [24]:
animal_div_period = pd.read_csv('./data/SQS_diversity_data/animal_diversity_period.csv')

## 2. Deal with missing data by interpolating

In [25]:
# Interpolate
time_new = np.linspace(1, 515, num=515) # interpolation time is chosen based on mean_ma

f1 = interp1d(data_skeletal_period['time'].to_numpy(),
              data_skeletal_period['mean'].to_numpy(), fill_value='extrapolate')

total_skeletal_mean_new = f1(time_new)

total_skeletal_mean_new = pd.DataFrame({'mean':total_skeletal_mean_new, 'time':time_new})
total_skeletal_mean_new['data'] = 'Skeletal biomass %'

# Interpolate
f2 = interp1d(sepkoski_period['mean_ma'].to_numpy(),
              sepkoski_period['diversity'].to_numpy(), fill_value='extrapolate')

sepkoski_diversity_new = f2(time_new)

sepkoski_diversity_new = pd.DataFrame({'mean':sepkoski_diversity_new, 'time':time_new})
sepkoski_diversity_new['data'] = 'Sepkoski diversity [genera]'

# Interpolate
f3 = interp1d(data_animal_period['time'].to_numpy(),
              data_animal_period['mean'].to_numpy(),fill_value='extrapolate')
total_animal_mean_new = f3(time_new)

total_animal_mean_new = pd.DataFrame({'mean':total_animal_mean_new, 'time':time_new})
total_animal_mean_new['data'] = 'Animal biomass %'

# Interpolate
f4 = interp1d(animal_div_period['mean_ma'].to_numpy(),
              animal_div_period['SQS_diversity'].to_numpy(), fill_value='extrapolate')

animal_diversity_new = f4(time_new)
animal_diversity_new = pd.DataFrame({'mean':animal_diversity_new, 'time':time_new})
animal_diversity_new['data'] = 'Animal diversity [genera]'


# 3. estimate and remove long term trend using a quadratic function

In [26]:
def quadratic_func(x, a, b, c):
    return a + b * x + c * x**2 
    
def quad_residuals(df, func):
    popt, pcov = curve_fit(func, df['time'],
                       df['mean'])
    df['quad_fit'] = func(df['time'], *popt)
    df['quad_residuals'] = df['mean'] - func(df['time'], *popt)
    
    return df

In [27]:
total_skeletal_mean_new = quad_residuals(total_skeletal_mean_new, quadratic_func)
sepkoski_diversity_new = quad_residuals(sepkoski_diversity_new, quadratic_func)
total_animal_mean_new = quad_residuals(total_animal_mean_new, quadratic_func)
animal_diversity_new = quad_residuals(animal_diversity_new, quadratic_func)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [28]:
fig = make_subplots(rows=4,cols=1,shared_xaxes=True, vertical_spacing=0.03,
                   subplot_titles=['<b>Skeletal Biomass %</b>', 
                                   '<b>Animal Biomass %</b>', 
                                   '<b>Sepkoski diversity [genera]</b>', 
                                   '<b>Animal diversity [genera]</b>'])

# Row 1
fig.add_trace(go.Scatter(x=total_skeletal_mean_new["time"], y=total_skeletal_mean_new["mean"],mode='lines',
                          line = dict(dash='solid', color='red'),
                        name='Mean'), row=1,col=1)
fig.add_trace(go.Scatter(x=total_skeletal_mean_new["time"], y=total_skeletal_mean_new["quad_fit"],mode='lines',
                          line = dict(dash='dot', color='red'),
                        name='Trend'), row=1,col=1)
fig.add_trace(go.Scatter(x=total_skeletal_mean_new["time"], y=total_skeletal_mean_new["quad_residuals"],mode='lines',
                          line = dict(dash='solid', color='blue'),
                        name='Residuals (mean - trend)'), row=1,col=1)

# row 2
fig.add_trace(go.Scatter(x=total_animal_mean_new["time"], y=total_animal_mean_new["mean"],mode='lines',
                          line = dict(dash='solid', color='red'),
                        showlegend=False), row=2,col=1)
fig.add_trace(go.Scatter(x=total_animal_mean_new["time"], y=total_animal_mean_new["quad_fit"],mode='lines',
                          line = dict(dash='dot', color='red'),
                        showlegend=False), row=2,col=1)
fig.add_trace(go.Scatter(x=total_animal_mean_new["time"], y=total_animal_mean_new["quad_residuals"],mode='lines',
                          line = dict(dash='solid', color='blue'),
                        showlegend=False), row=2,col=1)

# row 3
fig.add_trace(go.Scatter(x=sepkoski_diversity_new["time"], y=sepkoski_diversity_new["mean"],mode='lines',
                          line = dict(dash='solid', color='red'),
                        showlegend=False), row=3,col=1)
fig.add_trace(go.Scatter(x=sepkoski_diversity_new["time"], y=sepkoski_diversity_new["quad_fit"],mode='lines',
                          line = dict(dash='dot', color='red'),
                        showlegend=False), row=3,col=1)
fig.add_trace(go.Scatter(x=sepkoski_diversity_new["time"], y=sepkoski_diversity_new["quad_residuals"],mode='lines',
                          line = dict(dash='solid', color='blue'),
                        showlegend=False), row=3,col=1)

# row 4
fig.add_trace(go.Scatter(x=animal_diversity_new["time"], y=animal_diversity_new["mean"],mode='lines',
                          line = dict(dash='solid', color='red'),
                        showlegend=False), row=4,col=1)
fig.add_trace(go.Scatter(x=animal_diversity_new["time"], y=animal_diversity_new["quad_fit"],mode='lines',
                          line = dict(dash='dot', color='red'),
                        showlegend=False), row=4,col=1)
fig.add_trace(go.Scatter(x=animal_diversity_new["time"], y=animal_diversity_new["quad_residuals"],mode='lines',
                          line = dict(dash='solid', color='blue'),
                        showlegend=False), row=4,col=1)



fig.update_yaxes(matches=None, showline = True, 
                         mirror = True,linecolor = 'black',)
fig.update_xaxes(ticks='outside', showline = True, 
                         mirror = True,linecolor = 'black')
fig.add_hline(y=0, line=dict(dash='dash'))

fig.update_layout(height=1200, width=800,xaxis=dict(autorange='reversed'),paper_bgcolor = 'white',font_color = 'black',
                 plot_bgcolor = 'white',legend=dict(title="", orientation='h',font_size=14,
                yanchor="top",
                    y=1.06,
                    xanchor="left",
                    x=0.01))
fig.show()
fig.write_image('./figures/Figure_S11.jpeg', scale=6)

## 4. Get ARIMA residuals

In [10]:
def get_arima_residuals(df, order):
    model = ARIMA(df['quad_residuals'], order=order)
    results = model.fit()
    df['ARIMA_fit'] = results.fittedvalues
    df['ARIMA_residuals'] = results.resid

    return df

In [11]:
order = (2, 0, 0)  # AR(2) ARIMA model

total_animal_mean_new = get_arima_residuals(total_animal_mean_new,order)
total_skeletal_mean_new = get_arima_residuals(total_skeletal_mean_new,order)

animal_diversity_new = get_arima_residuals(animal_diversity_new,order)
sepkoski_diversity_new = get_arima_residuals(sepkoski_diversity_new,order)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## 5. Remove outliers

In [12]:
def find_outliers_iqr(data, column):
    q1 = data[column].quantile(0.01)
    q3 = data[column].quantile(0.99)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

In [13]:
outliers1 = find_outliers_iqr(sepkoski_diversity_new,'ARIMA_residuals')
outliers2 = find_outliers_iqr(total_skeletal_mean_new,'ARIMA_residuals')
outliers3 = find_outliers_iqr(total_animal_mean_new,'ARIMA_residuals')
outliers4 = find_outliers_iqr(animal_diversity_new,'ARIMA_residuals')

In [14]:
sepkoski = sepkoski_diversity_new.drop(index = list(set(outliers1.index).union(set(outliers2.index)))).copy()
skeletal_total = total_skeletal_mean_new.drop(index = list(set(outliers1.index).union(set(outliers2.index)))).copy()

In [15]:
animal_biomass = total_animal_mean_new.drop(index = list(set(outliers3.index).union(set(outliers4.index)))).copy()
animal_diversity = animal_diversity_new.drop(index = list(set(outliers3.index).union(set(outliers4.index)))).copy()

## 6. Calculate Spearman Correlation

In [19]:
# time_step = 10
for time_step in range(10,20,10):
    for df1,df2 in [(animal_biomass,animal_diversity), (skeletal_total, sepkoski)]:
        print(f"time step: {time_step}")
        print(f"{df1['data'].iloc[0]} & {df2['data'].iloc[0]}: {stats.spearmanr(df1['ARIMA_residuals'][::time_step], df2['ARIMA_residuals'][::time_step])}")

time step: 10
Animal biomass % & Animal diversity [genera]: SignificanceResult(statistic=0.6135746606334841, pvalue=1.6887027357996265e-06)
time step: 10
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Skeletal biomass % & Sepkoski diversity [genera]: SignificanceResult(statistic=0.6078715956629386, pvalue=1.7586973455335768e-06)
